In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf
import os
import scipy.optimize as optimize
import scipy.stats as stats
from scipy.stats import laplace_asymmetric

In [2]:
''' Data Generation '''
################################'Generate X and Y'##############################
################################ Xt ~ N(0,1)
################################ Yt = α + βYt-1 + γXt + εt + θεt-1
def generate_data(T, alpha_y, beta_y, gamma, theta, sigma_y, mu, seed=None):
    np.random.seed(seed)
    df = pd.DataFrame(index=range(T), columns=['x', 'y', 'epsilon_y'])
    x = np.random.normal(0, 1, T)
    y1 = np.random.normal(mu, sigma_y)
    epsilon_y = sigma_y*np.random.randn(T)
    df.iloc[0,:] = [x[0], y1, epsilon_y[0]]
    
    for i in range(0, T-1):
        df.iloc[i+1,0] = x[i+1] 
        # Xt ~ N(0,1)
        df.iloc[i+1,1] = alpha_y + beta_y * df.iloc[i,1] + gamma * df.iloc[i+1,0] + epsilon_y[i+1] + theta * epsilon_y[i] # generate Y(t) recursively
        # Yt = α + βYt-1 + γXt + εt + θεt-1
        df.iloc[i+1,2] = epsilon_y[i+1]
    
    return df

In [3]:
df = generate_data(1000, 0.2, 0.5, 0.3, 0.9, 1, 0, seed=42)
print(df)

            x         y epsilon_y
0    0.496714  1.399355  0.924634
1   -0.138264  1.749999   0.05963
2    0.647689  0.676037 -0.646937
3     1.52303  1.110908  0.698223
4   -0.234153  1.707094  0.393485
..        ...       ...       ...
995   -0.2811  2.353144 -0.026521
996  1.797687  1.010134 -0.881875
997  0.640843 -0.059434 -0.163067
998 -0.571179 -0.892734 -0.744903
999  0.572583 -1.420183 -0.675178

[1000 rows x 3 columns]


In [8]:
''' EM Algorithm'''
p = 0.4
M = 1000
n = M
T = 1000
df = df.astype('float64')  # convert data to float64 data type
y = df['y'].values
x = df['x'].values 
y_lag = np.roll(y, 1)
y_lag[0] = 0
y_lag = y_lag.reshape((-1, 1))
#X = np.column_stack((np.ones_like(x), y_lag, x))

τ_2 = 2 / p * (1 - p)
τ = np.sqrt(τ_2)
θ = (1 - 2 * p) / (p * (1 - p))

# Initial values
μ = np.empty(M)  # Empty array of length M
μ[0] = 1  # Set initial value μ_0 = 1
λ = np.empty(M)
λ[0] = 1
m = np.empty(M)
m[0] = 1
#Σ = np.zeros((3, 3))
α = np.empty(M)
α[0] = 1
γ = np.empty(M)
γ[0] = 1

mode_z = np.empty(M)
mode_σ = np.empty(M)
σ = np.empty(M)
β = np.zeros((M, 1))
print(β.shape)
z = np.zeros(M)
l = np.ones((1000, 1))
T = 1000  # Number of rows in your dataset

Σ = np.zeros((T, T))
X = np.zeros((T, 3))
X[:, 0] = np.ones(T)
X[:, 1] = y_lag[:, 0]
X[:, 2] = x

for i in range(M):
    # E-step: estimate mode_z
    mode_z[i] = μ[i] * ((1 + (9 * μ[i] ** 2) / (4 * λ[i] ** 2)) - (3 * μ[i]) / (4 * λ[i]))
    
    # Set mode_z = z_i and input in the parameters of the distribution of σ_i
    z[i] = mode_z[i] 
    α[i+1] = α[i] * (3 * n / 2)
    γ[i+1] = γ[i] + ((y[i+1] - x[i+1] * β[i]) ** 2) / (2 * z[i] * τ_2) + z[i]

    # Estimate the mode mode_σ_{i+1}
    mode_σ[i+1] = γ[i+1] / α[i+1] + 1

    # Set mode_σ_{i+1} = σ_{i+1} and compute U_{i+1}
    σ[i+1] = mode_σ[i+1]

    j = (σ[i+1] * τ_2 * z[i]) ** -1
    
    U = np.zeros((T, T))
    np.fill_diagonal(U, j)

    Σ = np.linalg.inv(X.T @ U @ X)

    r = θ / (σ[i+1] * τ_2)
    r_matr = np.zeros((3, 1))
    r_matr[0, 0] = r

    m[i+1] = Σ @ (X.T @ U @ y.reshape((-1, 1))).flatten() - (X.T @ U @ (r_matr @ l))
    β[i+1] = m[i+1, 0]

    μ[i+1] = np.sqrt(θ ** 2 + 2 * τ_2) / np.abs(y[i+1] - (x[i+1] * β[i + 1]))
    λ[i+1] = (θ ** 2 + 2 * τ_2) / (σ[i + 1] * τ_2)


print(β)

(1000, 1)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1000 is different from 1)

In [11]:
''' C'''
# Assuming your data is defined and initialized here
n = 1000
p = 0.6
df = df.astype('float64')
y = df['y'].values
x = df['x'].values
y = np.array(y)
x = np.array(x)

M = 10  # Number of iterations
# Initialize 
μ = 1
λ = 1
β = 1

τ_2 = 2 / p * (1 - p)
τ = np.sqrt(τ_2)
θ = (1 - 2 * p) / (p * (1 - p))

# EM Algorithm iterations
for i in range(M):
    if i == 0:
        # Use initial values μ_0, λ_0, m_0, Σ_0, α_0, γ_0
        pass
    else:
        # Use estimated β_{i-1} for computing μ_i and λ_i
        μ = np.sqrt(θ ** 2 + 2 * τ_2) / np.abs(y - x.T * β_)
        λ = (θ ** 2 + 2 * τ_2) / (σ * τ_2)
    
    y_lag = np.roll(y, 1)
    y_lag[0] = 0
    y_lag = y_lag.reshape((-1, 1))
    l = np.ones((len(x), 1))
    mode_z = μ * ((1 + (9 * μ ** 2) / (4 * λ ** 2)) - (3 * μ) / (4 * λ))
    z = mode_z
    α = (3 * n / 2)

    β_ = np.full(len(x), β)
    z_ = np.full(len(x), z)

    γ = np.sum(((y - x.T * β_ - θ * z_) ** 2) / (2 * z_ * τ_2)) + np.sum(z)
    mode_σ = γ / α + 1
    σ = mode_σ
    U = np.zeros((len(x), len(x)))
    u = (σ * τ_2 * z) ** -1
    np.fill_diagonal(U, u)
    
    r = θ / (σ * τ_2)
    l = l.reshape((-1, 1))
    X = np.zeros((len(x), len(x)))
    np.fill_diagonal(X, x.T)
    Σ = np.linalg.inv((X.T @ U) @ X)
    
    Σ_ = np.diag(Σ)
    Σ_ = Σ_.reshape((-1, 1))
    r = r.reshape((-1, 1))
    w = (X.T @ U @ y)
    w = w.reshape((-1, 1))

    print(w.shape)
    #m[i+1] = Σ @ (X.T @ U @ y) - ((r_matr) @ (X.T @ l))
    
    m = w - (r * (X.T @ l))
    print(Σ.shape)
    print(Σ_.shape)
    print((Σ_ * w).shape)
    print((X.T @ l).shape)
    print(r.shape)
    k = (X.T @ l) * r
    print(k.shape)

    #m[i+1] = Σ_ * w - k
    mkj = Σ_ * w - k
    
    β_ = m
    print(β_)


(1000, 1)
(1000, 1000)
(1000, 1)
(1000, 1)
(1000, 1)
(1, 1)
(1000, 1)
[[ 2.39826660e-01]
 [-7.34789274e-02]
 [ 2.47772105e-01]
 [ 6.74454000e-01]
 [-1.23045254e-01]
 [-1.40095628e-01]
 [ 1.16348233e+00]
 [ 6.11791294e-01]
 [-2.85422279e-01]
 [ 3.33425778e-01]
 [-3.02104623e-01]
 [-3.68431153e-01]
 [ 1.79326494e-01]
 [-1.18203888e+00]
 [-1.12695929e+00]
 [-2.30429930e-01]
 [-1.88949976e-01]
 [ 5.45848879e-02]
 [-2.17979084e-01]
 [-5.06788755e-01]
 [ 5.83645486e-01]
 [-6.47750895e-02]
 [ 4.13512496e-02]
 [-1.31911327e+00]
 [-4.83666121e-01]
 [ 7.55046212e-02]
 [-5.59382588e-01]
 [ 1.99419602e-01]
 [-2.08608866e-01]
 [-4.96946545e-02]
 [ 1.87384720e-02]
 [ 1.49589687e-01]
 [-4.74420856e-03]
 [-3.35993981e-01]
 [ 4.30276385e-01]
 [-4.99777601e-01]
 [ 6.17295311e-03]
 [ 9.05045681e-02]
 [-3.54597340e-01]
 [ 1.33455956e-01]
 [ 5.08288459e-01]
 [ 1.05512546e-01]
 [-6.99017647e-02]
 [-1.30406964e-01]
 [-6.33048592e-01]
 [-4.00934768e-01]
 [-1.35573242e-01]
 [ 3.92483762e-01]
 [ 2.56899697e-01]

ValueError: could not broadcast input array from shape (1000,1000) into shape (1000,)

In [7]:
''' C'''
# Assuming your data is defined and initialized here
n = 1000
p = 0.4
df = df.astype('float64')
y = df['y'].values
x = df['x'].values
y = np.array(y)
x = np.array(x)

M = 3  # Number of iterations
# Initialize 
μ = 1
λ = 1
β = 1

τ_2 = 2 / p * (1 - p)
τ = np.sqrt(τ_2)
θ = (1 - 2 * p) / (p * (1 - p))

# EM Algorithm iterations
for i in range(M):
    if i == 0:
        # Use initial values μ_0, λ_0, m_0, Σ_0, α_0, γ_0
        pass
    else:
        # Use estimated β_{i-1} for computing μ_i and λ_i
        μ = np.sqrt(θ ** 2 + 2 * τ_2) / np.abs(y - x.T * β)
        λ = (θ ** 2 + 2 * τ_2) / (σ * τ_2)
    
    y_lag = np.roll(y, 1)
    y_lag[0] = 0
    y_lag = y_lag.reshape((-1, 1))
    l = np.ones((len(x), 1))
    mode_z = μ * ((1 + (9 * μ ** 2) / (4 * λ ** 2)) - (3 * μ) / (4 * λ))
    z = mode_z
    α = (3 * n / 2)
    γ = np.sum(((y - x.T * β - θ * z) ** 2) / (2 * z * τ_2)) + np.sum(z)
    mode_σ = γ / α + 1
    σ = mode_σ
    U = np.zeros((len(x), len(x)))
    u = (σ * τ_2 * z) ** -1
    np.fill_diagonal(U, u)
    
    r = θ / (σ * τ_2)
    l = l.reshape((-1, 1))
    X = np.zeros((len(x), len(x)))
    np.fill_diagonal(X, x.T)
    Σ = np.linalg.inv((X.T @ U) @ X)
    
    Σ_ = np.diag(Σ)
    Σ_ = Σ_.reshape((-1, 1))
    r = r.reshape((-1, 1))
    w = (X.T @ U @ y)
    w = w.reshape((-1, 1))

    print(w.shape)
    #m[i+1] = Σ @ (X.T @ U @ y) - ((r_matr) @ (X.T @ l))
    
    m = w - (r * (X.T @ l))
    print(Σ.shape)
    print(Σ_.shape)
    print((Σ_ * w).shape)
    print((X.T @ l).shape)
    print(r.shape)
    k = (X.T @ l) * r
    print(k.shape)

    #m[i+1] = Σ_ * w - k
    mkj = Σ_ * w - k
    
    β = mkj
print(β)

(1000, 1)
(1000, 1000)
(1000, 1)
(1000, 1)
(1000, 1)
(1, 1)
(1000, 1)
(1000, 1)
(1000, 1000)
(1000, 1)
(1000, 1)
(1000, 1)
(1, 1)
(1000, 1)


/var/folders/f2/mjfy7nsn0tqbsmsk74mmlybh0000gn/T/ipykernel_23508/501196479.py:28: RuntimeWarning: divide by zero encountered in divide
  μ = np.sqrt(θ ** 2 + 2 * τ_2) / np.abs(y - x.T * β)
/var/folders/f2/mjfy7nsn0tqbsmsk74mmlybh0000gn/T/ipykernel_23508/501196479.py:35: RuntimeWarning: invalid value encountered in subtract
  mode_z = μ * ((1 + (9 * μ ** 2) / (4 * λ ** 2)) - (3 * μ) / (4 * λ))


(1000, 1)
(1000, 1000)
(1000, 1)
(1000, 1)
(1000, 1)
(1, 1)
(1000, 1)
[[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]